In [ ]:
import pandas as pd
import duckdb
import numpy as np

In [ ]:
con = duckdb.connect(database=':memory:', read_only=False)
con.execute("CREATE TABLE ist AS SELECT * FROM read_csv_auto('IST_corrected.csv')")
raw_size = con.execute("SELECT * FROM ist").fetchdf().size
print(raw_size)

In [ ]:
seperate_df_columns = "AGE, SEX, RSBP, STYPE"

# Strings of comma seperated column names (variables) to be selected from the table, subcaterogized in determined time.
randomisation = "RDEF1, RDEF2, RDEF3, RDEF4, RDEF5, RDEF6, RDEF7, RDEF8, RXASP, RXHEP"
# fourteen_day_form is unused
fourteen_day_form = "DSCH, DIVH, DAP, DOAC, DGORM, DSTER, DCAA, DHAEMD, DCAREND, DTHROMB, DMAJNCH"

final_diagnosis = "DDIAGISC, DDIAGHA, DDIAGUN, DNOSTRK"
recurrent_stroke = "DRSISC, DRSH"
fourteen_day_death = "DPE, DDEAD"
six_months = "OCCODE"
# Join column String's
columns = ', '.join([randomisation, final_diagnosis, recurrent_stroke, fourteen_day_death, seperate_df_columns, six_months])
# Construct dataframe
df = con.execute("SELECT {} FROM ist".format(columns)).fetchdf()
# df_seperate = con.execute("SELECT {0}, {1} FROM ist".format(seperate_df_columns, six_months)).fetchdf()
# Keep track of pre-pre-processed data size
var_list = columns.split(", ")
print(var_list)
initial_size = df.size
print(initial_size)

In [ ]:
dtypes_before = df.dtypes
print(dtypes_before)

In [ ]:
df.info

In [ ]:
def pre_process_columns(some_df, column):
    if str(column) in ['age', 'rsbp', 'occode']:
        return
    some_df[column] = some_df[column].apply(lambda x : str(x).upper() if type(x) == object else x)
    if str(column) in ['sex', 'stype']:
        return
    some_df[column] = some_df[column].replace({'Y': 1, 'M': 1, 'H': 1, 'L': 1, 'N': 0, 'C' : np.nan, 'U' : np.nan})

In [ ]:
for col in df.columns:
    pre_process_columns(df, col)

In [ ]:
df.info

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df = df.dropna(axis = 0, how = 'any')

In [ ]:
# Y -> 1
# N -> 0
# Set all types with (1, 0) to int
replace_vars = ', '.join([randomisation, final_diagnosis, recurrent_stroke, fourteen_day_death]).split(', ')
for col in replace_vars:
    df[col.lower()] = df[col.lower()].astype(int, copy=False)

In [ ]:
df.dtypes

In [ ]:
df.info

In [ ]:
df.to_csv('IST_corrected_processed.csv',index=False)

In [ ]:
df['age'].mean()

In [ ]:
df['rsbp'].mean()

In [ ]:
df['rdef1'].mean()